In [ ]:
import const as CONST
import matplotlib.pyplot as plt
from darts.metrics import mape
from darts import TimeSeries
import pandas as pd
import const as CONST
import warnings
import os
from run import MODEL_CONFIGS 

warnings.simplefilter(action="ignore", category=FutureWarning)

from darts import concatenate
from datasets import load_datasets

In [ ]:
from view_results import load_results


ds = load_datasets()

for config in mo
predictions = load_results(config)
for prediction, ticker in zip(predictions, ds.original.used_tickers):
    if config.output_len == 1:
        result = concatenate(prediction, 0)
        result.to_csv(f"{config.result_path}/{ticker}.csv")

